In [ ]:
import pandas as pd
df = pd.read_csv("../data/processed/telco_customer_churn_cleaned.csv") 


In [17]:
# Columns that always cause leakage
leakage_cols = [
    "churn_label", 
    "churn_score", 
    "cltv", 
    "churn_reason",
    "count"
]

# Columns that add no predictive value
drop_geo = [
    "country", "state", "city", "zip_code",
    "lat_long", "latitude", "longitude"
]

df_ml = df.drop(columns=leakage_cols + drop_geo)


In [18]:
y = df_ml["churn_value"]
x = df_ml.drop(columns=["churn_value", "customerid"])

x = pd.get_dummies(x, drop_first=True)



In [ ]:
x.to_csv("../data/processed/telco_features.csv", index=False)
y.to_csv("../data/processed/telco_target.csv", index=False)



Index(['tenure_months', 'monthly_charges', 'total_charges', 'gender_Male',
       'senior_citizen_Yes', 'partner_Yes', 'dependents_Yes',
       'phone_service_Yes', 'multiple_lines_No phone service',
       'multiple_lines_Yes', 'internet_service_Fiber optic',
       'internet_service_No', 'online_security_No internet service',
       'online_security_Yes', 'online_backup_No internet service',
       'online_backup_Yes', 'device_protection_No internet service',
       'device_protection_Yes', 'tech_support_No internet service',
       'tech_support_Yes', 'streaming_tv_No internet service',
       'streaming_tv_Yes', 'streaming_movies_No internet service',
       'streaming_movies_Yes', 'contract_One year', 'contract_Two year',
       'paperless_billing_Yes', 'payment_method_Credit card (automatic)',
       'payment_method_Electronic check', 'payment_method_Mailed check'],
      dtype='object')
churn_value
